# 🔌 Plugin Development

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artiso-ai/dppvalidator/blob/main/examples/notebooks/05_plugin_development.ipynb)

Learn how to create reusable validation plugins for dppvalidator.

This notebook covers:
- Plugin architecture overview
- Creating an industry-specific plugin
- Packaging and distributing plugins

In [ ]:
!pip install -q dppvalidator

## 🏗️ Plugin Architecture

dppvalidator plugins extend the validation engine with:

- **Custom Rules** - Industry-specific validation logic
- **Vocabularies** - Controlled value sets
- **Exporters** - Custom output formats

```
Plugin Structure:
├── __init__.py        # Plugin entry point
├── rules.py           # Custom validation rules
├── vocabularies/      # Industry vocabularies
└── exporters.py       # Custom exporters (optional)
```

## 🧵 Example: Textile Industry Plugin

Let's create a plugin for textile/fashion industry validation:

In [ ]:
from typing import Literal

# ISO 2076 Fiber Codes (subset)
ISO_2076_FIBERS = {
    "CO": "Cotton",
    "WO": "Wool",
    "SE": "Silk",
    "LI": "Linen",
    "PES": "Polyester",
    "PA": "Polyamide (Nylon)",
    "EL": "Elastane",
    "VI": "Viscose",
    "CLY": "Lyocell",
}


class FiberCodeRule:
    """TXT001: Material names should use ISO 2076 fiber codes."""

    rule_id = "TXT001"
    description = "Fiber names should follow ISO 2076 codes"
    severity: Literal["error", "warning", "info"] = "warning"
    suggestion = "Use ISO 2076 fiber codes (e.g., 'CO' for Cotton, 'PES' for Polyester)"
    docs_url = "https://artiso-ai.github.io/dppvalidator/plugins/textile/TXT001"

    def check(self, passport) -> list[tuple[str, str]]:
        violations = []

        if not passport.credential_subject:
            return violations

        materials = passport.credential_subject.materials_provenance
        if not materials:
            return violations

        for i, material in enumerate(materials):
            name = material.name or ""
            # Check if name contains a valid fiber code
            has_valid_code = any(
                code in name.upper() or fiber.lower() in name.lower()
                for code, fiber in ISO_2076_FIBERS.items()
            )

            if not has_valid_code and name:
                violations.append(
                    (
                        f"$.credentialSubject.materialsProvenance[{i}].name",
                        f"Material '{name}' does not match ISO 2076 fiber codes",
                    )
                )

        return violations


print("✓ TXT001 rule defined")

In [ ]:
class BlendedFiberRule:
    """TXT002: Blended fabrics must list all component fibers."""

    rule_id = "TXT002"
    description = "Blended fabrics must list all fiber components"
    severity: Literal["error", "warning", "info"] = "warning"
    suggestion = "For blended fabrics, list each fiber separately with mass fractions"
    docs_url = "https://artiso-ai.github.io/dppvalidator/plugins/textile/TXT002"

    def check(self, passport) -> list[tuple[str, str]]:
        violations = []

        if not passport.credential_subject:
            return violations

        materials = passport.credential_subject.materials_provenance
        if not materials:
            return violations

        for i, material in enumerate(materials):
            name = (material.name or "").lower()
            # Detect blend keywords
            blend_keywords = ["blend", "mix", "/", "+"]
            if any(kw in name for kw in blend_keywords):
                violations.append(
                    (
                        f"$.credentialSubject.materialsProvenance[{i}]",
                        f"Blended material '{material.name}' should be split into separate entries",
                    )
                )

        return violations


print("✓ TXT002 rule defined")

In [ ]:
class CareInstructionRule:
    """TXT003: Textile products should have care instructions."""

    rule_id = "TXT003"
    description = "Textile products should include care instructions"
    severity: Literal["error", "warning", "info"] = "info"
    suggestion = "Add care instructions following ISO 3758 care labeling codes"
    docs_url = "https://artiso-ai.github.io/dppvalidator/plugins/textile/TXT003"

    def check(self, passport) -> list[tuple[str, str]]:
        violations = []

        if not passport.credential_subject:
            return violations

        # Check for care instructions in claims
        claims = passport.credential_subject.conformity_claim or []
        has_care_info = any("care" in str(getattr(c, "topic", "")).lower() for c in claims)

        if not has_care_info:
            violations.append(
                (
                    "$.credentialSubject.conformityClaim",
                    "Consider adding care instructions for textile products",
                )
            )

        return violations


print("✓ TXT003 rule defined")

## 📦 Creating the Plugin Class

In [ ]:
class TextilePlugin:
    """Textile industry validation plugin for dppvalidator."""

    name = "dppvalidator-textile"
    version = "1.0.0"

    def __init__(self):
        self.rules = [
            FiberCodeRule(),
            BlendedFiberRule(),
            CareInstructionRule(),
        ]
        self.vocabularies = {
            "iso_2076_fibers": ISO_2076_FIBERS,
        }

    def get_rules(self):
        """Return all validation rules from this plugin."""
        return self.rules

    def get_vocabularies(self):
        """Return controlled vocabularies."""
        return self.vocabularies


plugin = TextilePlugin()
print(f"✓ {plugin.name} v{plugin.version}")
print(f"  Rules: {len(plugin.rules)}")
print(f"  Vocabularies: {len(plugin.vocabularies)}")

## 🔌 Using the Plugin

In [ ]:
from dppvalidator.validators import ValidationEngine
from dppvalidator.validators.rules import ALL_RULES
from dppvalidator.validators.semantic import SemanticValidator

# Combine core rules with plugin rules
textile_rules = ALL_RULES + plugin.get_rules()

# Create engine with textile plugin
engine = ValidationEngine()
engine._semantic_validator = SemanticValidator(rules=textile_rules)

print(f"✓ Engine configured with {len(textile_rules)} total rules")

In [ ]:
# Test with a textile DPP
textile_dpp = {
    "type": ["DigitalProductPassport", "VerifiableCredential"],
    "@context": [
        "https://www.w3.org/ns/credentials/v2",
        "https://test.uncefact.org/vocabulary/untp/dpp/0.6.1/",
    ],
    "id": "https://example.com/credentials/textile-001",
    "issuer": {"id": "https://example.com/issuers/001", "name": "Fashion Brand"},
    "credentialSubject": {
        "type": ["Product"],
        "id": "https://example.com/products/001",
        "name": "Premium T-Shirt",
        "materialsProvenance": [
            {
                "type": ["Material"],
                "name": "Cotton/Polyester Blend",  # Will trigger TXT002
                "massFraction": 1.0,
                "hazardous": False,
            }
        ],
    },
}

result = engine.validate(textile_dpp)

print(f"Valid: {result.valid}")
print(f"Warnings: {result.warning_count}")
print(f"Info: {len(result.info)}")

print("\nTextile Plugin Findings:")
for issue in result.warnings + result.info:
    if issue.code.startswith("TXT"):
        print(f"  [{issue.code}] {issue.message}")
        if issue.suggestion:
            print(f"    💡 {issue.suggestion}")

## 📦 Packaging as a Distributable Plugin

To distribute your plugin via PyPI:

```
dppvalidator-textile/
├── pyproject.toml
├── README.md
└── src/
    └── dppvalidator_textile/
        ├── __init__.py
        ├── rules.py
        └── vocabularies/
            └── iso_2076.json
```

Example `pyproject.toml`:

```toml
[project]
name = "dppvalidator-textile"
version = "1.0.0"
dependencies = ["dppvalidator>=0.1.0"]

[project.entry-points."dppvalidator.plugins"]
textile = "dppvalidator_textile:TextilePlugin"
```

## 🎉 Key Takeaways

- Plugins extend dppvalidator with industry-specific rules
- Rules follow a simple protocol: `rule_id`, `severity`, `check()`
- Bundle vocabularies with your plugin
- Distribute via PyPI using entry points